# Fossil fuel imports and exports to South Africa, Nigeria and Egypt
This notebook calculates the imports and exports of fossil fuels from named countries, using data from https://comtrade.un.org/data. We use the designation "from" to indicate data from the country we are considering, and "to" to reference data originating from other countries - this should not be confused with import and export data, which are found in both datasets. 

Note that references to US are using a conversion database that originated in the USA, not simply accounting for trade between the USA and the country in question. 

In [1]:
import numpy as np
import pandas as pd
from os.path import join

In [2]:
# Read data
input_folder = "input_data"
output_folder = "output"
plastic_file = "world_plastic_trade.csv"
file_to = "AllTradeTo3Countries.csv"
file_from = "AllTradeFrom3Countries.csv"
input_to = pd.read_csv(join(input_folder, file_to))
input_from = pd.read_csv(join(input_folder, file_from))
plastic_data = pd.read_csv(join(input_folder, plastic_file))

Intial work indicated that data from 2018 seems to be more balanced than data from 2019 in terms of having both sides of the import and export by countries that should be in a reciprocal relationship. We will use that henceforth. 2020 is presumably anomalous. 

In [3]:
year = 2018
countries = ["South Africa", "Nigeria", "Egypt"]
impex = ["Import", "Export"]
plastic_waste = "Waste, parings and scrap, of plastics"
report_cols = ["Reporter", "Trade Flow", "Netweight (kg)"]
composite = ["Coal; briquettes, ovoids and similar solid fuels manufactured from coal"]
redundant_cols = [
    "Classification", "Period", "Period Desc.", "Reporter Code", 
    "FOB Trade Value (US$)", "CIF Trade Value (US$)", 'Mode of Transport Code',
    "Alt Qty", "Alt Qty Unit Code", "Alt Qty Unit", 'Qty Unit Code', 
    "Customs", "Mode of Transport", "Customs Proc. Code", "Gross weight (kg)",
    "Qty Unit", "Qty"
]
from_year = input_from[input_from.Year==year][[col for col in input_from.columns if col not in redundant_cols]]
to_year = input_to[input_to.Year==year][[col for col in input_from.columns if col not in redundant_cols]]
plastic_year = plastic_data[plastic_data.Year==year][[col for col in plastic_data if col not in redundant_cols]]
assert set(from_year.columns) == set(to_year.columns)

In [4]:
plastic_year[["Reporter", "Trade Flow"]]

,Reporter,Trade Flow
0,Albania,Import
1,Albania,Export
2,Andorra,Import
3,Andorra,Export
4,Andorra,Re-Import
...,...,...
708,Libya,Import
709,Libya,Export
710,Barbados,Import
711,Barbados,Export


In [5]:
plastic_report = plastic_year[
    plastic_year["Reporter"].isin(countries) & plastic_year["Trade Flow"].isin(impex) 
    & plastic_year.Commodity.isin([plastic_waste])
][report_cols]
plastic_report = plastic_report.pivot(columns="Trade Flow", index=["Reporter"], values="Netweight (kg)")
plastic_report["Net Import"] = plastic_report.Import - plastic_report.Export
plastic_report

Trade Flow,Export,Import,Net Import
Reporter,,,
Egypt,2999.0,19889853.0,19886854.0
Nigeria,2164750.0,16612038.0,14447288.0
South Africa,11389890.0,18074903.0,6685013.0


In [6]:
unavailable_rows = from_year[from_year.Year == 0]
nw = "Netweight (kg)"
world_sum_exp = pd.DataFrame(columns=("Country","Commodity", "ExportsFrom", "World", "ImportsTo", "ReExportsFrom"))
world_sum_imp = pd.DataFrame(columns=("Country","Commodity", "ImportsFrom", "World", "ReImportsFrom", "ExportsTo"))
for country in countries:
    for commodity in from_year.Commodity.unique():
        exports_from = from_year[(from_year["Trade Flow"] == "Export") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        reexports_from = from_year[(from_year["Trade Flow"] == "Re-Export") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        reimports_from = from_year[(from_year["Trade Flow"] == "Re-Import") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        imports_to = to_year[(to_year["Trade Flow"] == "Import") & (to_year.Commodity == commodity) & (to_year.Partner == country)]
        exports_to = to_year[(to_year["Trade Flow"] == "Export") & (to_year.Commodity == commodity) & (to_year.Partner == country)]
        imports_from = from_year[(from_year["Trade Flow"] == "Import") & (from_year.Commodity == commodity) & (from_year.Reporter == country)]
        assert len(exports_from[exports_from.Partner=="World"][nw]) in [0, 1]
        world_sum_exp = world_sum_exp.append(pd.DataFrame({
            "Country": country,
            "Commodity": commodity, 
            "ExportsFrom": np.nansum(exports_from[exports_from.Partner!="World"][nw]),
            "World": exports_from[exports_from.Partner=="World"][nw],
            "ImportsTo": np.nansum(imports_to[imports_to.Partner!="World"][nw]), 
            "ReExportsFrom": np.nansum(reexports_from[reexports_from.Partner!="World"][nw]) 
        }))
        assert len(imports_from[imports_from.Partner=="World"][nw]) in [0, 1]
        world_sum_imp = world_sum_imp.append(pd.DataFrame({
            "Country": country,
            "Commodity": commodity, 
            "ImportsFrom": np.nansum(imports_from[imports_from.Partner!="World"][nw]), 
            "ReImportsFrom": np.nansum(reimports_from[reimports_from.Partner!="World"][nw]), 
            "World": np.nansum(imports_from[imports_from.Partner=="World"][nw]),
            "ExportsTo": np.nansum(exports_to[exports_to.Partner!="World"][nw])
        }, index=[1]))

In [7]:
# We need to avoid a zero division error
world_sum_imp["ImpFromExpToRatio"] = world_sum_imp.ImportsFrom / (world_sum_imp.ExportsTo + 1e-20)

In [8]:
world_sum_imp

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio
1,South Africa,"Coal; briquettes, ovoids and similar solid fue...",3.051784e+09,3.051784e+09,300727000.0,1.713781e+09,1.780732e+00
1,South Africa,"Coal; anthracite, whether or not pulverised, b...",3.542647e+08,3.542647e+08,0.0,3.681602e+07,9.622568e+00
1,South Africa,"Coal; bituminous, whether or not pulverised, b...",1.273978e+09,1.273978e+09,0.0,1.470915e+09,8.661127e-01
1,South Africa,"Coal; (other than anthracite and bituminous), ...",1.379110e+09,1.379110e+09,300727000.0,1.483721e+08,9.294940e+00
1,South Africa,"Briquettes, ovoids and similar solid fuels; ma...",4.443221e+07,4.443221e+07,0.0,5.693384e+07,7.804183e-01
1,South Africa,"Lignite; whether or not agglomerated, excludin...",1.350000e+02,1.350000e+02,0.0,2.503600e+07,5.392235e-06
1,South Africa,"Peat; (including peat litter), whether or not ...",5.674315e+07,5.674316e+07,15.0,5.718241e+07,9.923183e-01
1,South Africa,"Coke and semi-coke; of coal, lignite or peat, ...",8.016025e+08,8.016025e+08,68000.0,7.490820e+08,1.070113e+00
1,South Africa,"Coal gas, water gas, producer gas and similar ...",1.533000e+03,1.533000e+03,0.0,0.000000e+00,1.533000e+23
1,South Africa,"Tar distilled from coal, from lignite, peat an...",1.217796e+06,1.217796e+06,0.0,1.324525e+06,9.194209e-01


In [9]:
world_sum_exp["ExpFromImpToRatio"] = world_sum_exp.ExportsFrom / world_sum_exp.ImportsTo

In [10]:
world_sum_exp

,Country,Commodity,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio
1,South Africa,"Coal; briquettes, ovoids and similar solid fue...",8.131101e+10,8.131101e+10,6.569519e+10,205906.0,1.237701e+00
121,South Africa,"Coal; anthracite, whether or not pulverised, b...",1.240604e+09,1.240604e+09,1.077735e+09,200522.0,1.151121e+00
174,South Africa,"Coal; bituminous, whether or not pulverised, b...",7.892341e+10,7.892341e+10,2.583692e+10,5384.0,3.054675e+00
242,South Africa,"Coal; (other than anthracite and bituminous), ...",1.016501e+09,1.016501e+09,3.865228e+10,0.0,2.629861e-02
302,South Africa,"Briquettes, ovoids and similar solid fuels; ma...",1.304936e+08,1.304936e+08,5.505881e+07,0.0,2.370076e+00
333,South Africa,"Lignite; whether or not agglomerated, excludin...",1.200000e+01,1.200000e+01,2.660000e+02,0.0,4.511278e-02
338,South Africa,"Peat; (including peat litter), whether or not ...",4.107520e+05,4.107550e+05,1.660503e+06,220204.0,2.473660e-01
379,South Africa,"Coke and semi-coke; of coal, lignite or peat, ...",1.068653e+08,1.068653e+08,3.005672e+08,0.0,3.555454e-01
420,South Africa,"Coal gas, water gas, producer gas and similar ...",1.993100e+04,1.993200e+04,2.594400e+04,0.0,7.682316e-01
436,South Africa,"Tar distilled from coal, from lignite, peat an...",3.756630e+05,3.756660e+05,1.193550e+05,15.0,3.147443e+00


In [11]:
# Calculate the means to establish re-imports/re-exports are only small fractions of the total
np.mean(world_sum_exp.ReExportsFrom / world_sum_exp.ExportsFrom)

0.013962310730955194

In [12]:
np.mean(world_sum_imp.ReImportsFrom / world_sum_imp.ImportsFrom)

0.006222370213635653

In [13]:
# Calculate the geometric mean, using only values that are within two orders of magnitude
def geomean_trunc(x):
    y = np.log(x[np.isfinite(x) & (x<100) & (x>0.01)])
    return np.exp(np.mean(y))
def geomean(x):
    y = np.log(x[(x != 0) & np.isfinite(x)])
    return np.exp(np.mean(y))
def logstdev_trunc(x):
    y = np.log(x[np.isfinite(x) & (x<100) & (x>0.01)])
    return np.exp(np.std(y))

In [14]:
geomean(world_sum_exp.ExpFromImpToRatio)

0.8822072820101641

In [15]:
geomean_trunc(world_sum_exp.ExpFromImpToRatio)

0.6737632368057778

In [16]:
logstdev_trunc(world_sum_exp.ExpFromImpToRatio)

4.615138425829305

In [17]:
geomean(world_sum_imp.ImpFromExpToRatio)

29.835878444272478

In [18]:
geomean_trunc(world_sum_imp.ImpFromExpToRatio)

1.3199080885031451

In [19]:
logstdev_trunc(world_sum_imp.ImpFromExpToRatio)

6.038950748673212

# Begin conversion into emissions equivalents
We use data from https://ghgprotocol.org/calculation-tools#cross_sector_tools_id to perform the conversion. This is in turn based on http://www.ipcc-nggip.iges.or.jp/public/2006gl/vol2.html data, but converted into units of emissions/tonne rather than emissions/TJ. We compare these numbers when applied to coal with the numbers using the USA data, found in https://www.epa.gov/sites/production/files/2020-04/documents/ghg-emission-factors-hub.pdf
to ensure that the values are reasonable. 
## USA analysis

In [20]:
us_folder = "USA_data"
solid_factor_file = "solidFuelsConversionFactors.csv"
solid_factors = pd.read_csv(join(input_folder, us_folder, solid_factor_file), index_col=0)
kgInShortTon = 907.18
solid_factors = solid_factors.append(pd.DataFrame({c: np.nan for c in solid_factors.columns}, index=["nan"]))
solid_factors

,mmBtu per short ton,kg CO2 per mmBtu,g CH4 per mmBtu,g N2O per mmBtu,kg CO2 per short ton,g CH4 per short ton,g N2O per short ton
Anthracite Coal,25.09,103.69,11.0,1.6,2602.0,276.0,40.0
Bituminous Coal,24.93,93.28,11.0,1.6,2325.0,274.0,40.0
Sub-bituminous Coal,17.25,97.17,11.0,1.6,1676.0,190.0,28.0
Lignite Coal,14.21,97.72,11.0,1.6,1389.0,156.0,23.0
Mixed Coal (Commercial Sector),21.39,94.27,11.0,1.6,2016.0,235.0,34.0
Mixed Coal (Electric Power Sector),19.73,95.52,11.0,1.6,1885.0,217.0,32.0
Mixed Coal (Industrial Coking),26.28,93.90,11.0,1.6,2468.0,289.0,42.0
Mixed Coal (Industrial Sector),22.35,94.67,11.0,1.6,2116.0,246.0,36.0
Coal Coke,24.80,113.67,11.0,1.6,2819.0,273.0,40.0
Municipal Solid Waste,9.95,90.70,32.0,4.2,902.0,318.0,42.0


In [21]:
# Work out how these names correspond to the comtrade names
input_from.Commodity.unique()

array(['Coal; briquettes, ovoids and similar solid fuels manufactured from coal',
       'Coal; anthracite, whether or not pulverised, but not agglomerated',
       'Coal; bituminous, whether or not pulverised, but not agglomerated',
       'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated',
       'Briquettes, ovoids and similar solid fuels; manufactured from coal',
       'Lignite; whether or not agglomerated, excluding jet',
       'Peat; (including peat litter), whether or not agglomerated',
       'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon',
       'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons',
       'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated or partially distilled; including reconstituted tars',
       'Oils and other products of the distillation of high temperature coal t

In [22]:
solid_name_conversion = {
    'Coal; anthracite, whether or not pulverised, but not agglomerated': "Anthracite Coal", 
    'Coal; bituminous, whether or not pulverised, but not agglomerated': "Bituminous Coal",
    'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated': "Sub-bituminous Coal",
    'Briquettes, ovoids and similar solid fuels; manufactured from coal': "Mixed Coal (Commercial Sector)",
    'Lignite; whether or not agglomerated, excluding jet': "Lignite Coal",
    'Peat; (including peat litter), whether or not agglomerated': "Peat",
    'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon': "Coal Coke",
}
world_sum_exp = world_sum_exp
world_sum_exp["emissionsCommodity"] = [
    solid_name_conversion[v]  if v in solid_name_conversion.keys() else "nan" for v in world_sum_exp.Commodity
]
world_sum_imp["emissionsCommodity"] = [
    solid_name_conversion[v]  if v in solid_name_conversion.keys() else "nan" for v in world_sum_imp.Commodity
]
# Apply emissions conversion factors, including converting kg CO2 to tonnes Co2
world_sum_exp["tCO2_USA"] = solid_factors[
    "kg CO2 per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000 
world_sum_exp["tCH4_USA"] = solid_factors[
    "g CH4 per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000000
world_sum_exp["tN2O_USA"] = solid_factors[
    "g N2O per short ton"
][world_sum_exp["emissionsCommodity"]].values * world_sum_exp.ExportsFrom / kgInShortTon / 1000000
# And to imports
world_sum_imp["tCO2_USA"] = solid_factors[
    "kg CO2 per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000 
world_sum_imp["tCH4_USA"] = solid_factors[
    "g CH4 per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000000
world_sum_imp["tN2O_USA"] = solid_factors[
    "g N2O per short ton"
][world_sum_imp["emissionsCommodity"]].values * world_sum_imp.ImportsFrom / kgInShortTon / 1000000

## GHG protocol analysis

In [23]:
ghgprotocol_folder = "ghg_protocol_UN_data"
co2_file = "CO2_emissions_per_mass_ghgprotocol.csv"
ch4_file = "CH4_emissions_per_mass_ghgprotocol.csv"
n2o_file = "N2O_emissions_per_mass_ghgprotocol.csv"
co2_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, co2_file), index_col=0)
ch4_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, ch4_file), index_col=0)
n2o_factors = pd.read_csv(join(input_folder, ghgprotocol_folder, n2o_file), index_col=0)
co2_factors=co2_factors.append(
    pd.DataFrame({c: 0 for c in co2_factors.columns}, index=["Unburned"])
)
ch4_factors=ch4_factors.append(
    pd.DataFrame({c: 0 for c in ch4_factors.columns}, index=["Unburned"])
)
n2o_factors=n2o_factors.append(
    pd.DataFrame({c: 0 for c in n2o_factors.columns}, index=["Unburned"])
)
ch4_factors

,TJ/Gg,kg/TJ,kg/tonne
Crude oil,42.30,10,0.42300
Orimulsion,27.50,10,0.27500
Natural Gas Liquids,44.20,10,0.44200
Motor gasoline,44.30,10,0.44300
Aviation gasoline,44.30,10,0.44300
Jet gasoline,44.30,10,0.44300
Jet kerosene,44.10,10,0.44100
Other kerosene,43.80,10,0.43800
Shale oil,38.10,10,0.38100
Gas/Diesel oil,43.00,10,0.43000


In [24]:
world_sum_exp.Commodity.unique()

array(['Coal; briquettes, ovoids and similar solid fuels manufactured from coal',
       'Coal; anthracite, whether or not pulverised, but not agglomerated',
       'Coal; bituminous, whether or not pulverised, but not agglomerated',
       'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated',
       'Briquettes, ovoids and similar solid fuels; manufactured from coal',
       'Lignite; whether or not agglomerated, excluding jet',
       'Peat; (including peat litter), whether or not agglomerated',
       'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon',
       'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons',
       'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated or partially distilled; including reconstituted tars',
       'Oils and other products of the distillation of high temperature coal t

In [25]:
ghg_factors_conversion = {
    'Coal; anthracite, whether or not pulverised, but not agglomerated': "Anthracite",
    'Coal; bituminous, whether or not pulverised, but not agglomerated': "Bitumen",
    'Coal; (other than anthracite and bituminous), whether or not pulverised but not agglomerated': "Sub bituminous coal",
    'Briquettes, ovoids and similar solid fuels; manufactured from coal': "Other bituminous coal",
    'Lignite; whether or not agglomerated, excluding jet': "Lignite",
    'Peat; (including peat litter), whether or not agglomerated': "Peat",
    'Coke and semi-coke; of coal, lignite or peat, whether or not agglomerated; retort carbon': "Coking coal",
    'Coal gas, water gas, producer gas and similar gases, other than petroleum gases and other gaseous hydrocarbons': "Gas coke",
    'Tar distilled from coal, from lignite, peat and other mineral tars, whether or not dehydrated '
    'or partially distilled; including reconstituted tars': "Coal tar",
    'Oils and other products of the distillation of high temperature coal tar; similar products in which the weight '
    'of the aromatic constituents exceeds that of the non-aromatic constituents': "Other petroleum products",
    'Pitch and pitch coke; obtained from coal tar or from other mineral tars': "Coke oven coke",
    'Petroleum oils and oils obtained from bituminous minerals; crude': "Crude oil",
    'Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% '
    'or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of '
    'the preparations; waste oils': "Motor gasoline",
    'Petroleum gases and other gaseous hydrocarbons': "Refinery gas",
    'Petroleum jelly; paraffin wax, micro-crystalline petroleum wax, slack wax, ozokerite, lignite wax, '
    'peat wax, other mineral waxes, similar products obtained by synthesis, other processes; coloured or not': "Unburned",
    'Petroleum coke, petroleum bitumen; other residues of petroleum oils or oils obtained from '
    'bituminous minerals': "Petroleum coke",
    'Bitumen and asphalt, natural; bituminous or oil shale and tar sands; asphaltites '
    'and asphaltic rocks': "Oil shale and tar sands",
    'Bituminous mixtures based on natural asphalt; on natural bitumen, on petroleum bitumen, on mineral '
    'tar or on mineral tar pitch (e.g. bituminous mastics, cut-backs)': "Unburned"
}

In [26]:
world_sum_exp["ghgCategory"] = [
    ghg_factors_conversion[v]  if v in ghg_factors_conversion.keys() else "nan" for v in world_sum_exp.Commodity
]
world_sum_imp["ghgCategory"] = [
    ghg_factors_conversion[v]  if v in ghg_factors_conversion.keys() else "nan" for v in world_sum_imp.Commodity
]
# We want to remove the aggregate values to prevent confusion:
world_sum_exp = world_sum_exp[~world_sum_exp.Commodity.isin(composite)]
world_sum_imp = world_sum_imp[~world_sum_imp.Commodity.isin(composite)]

In [27]:
# Convert from kg product to tonnes, then kg CO2 to tonnes
world_sum_exp["tCO2"] = world_sum_exp.ExportsFrom * co2_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tCH4"] = world_sum_exp.ExportsFrom * ch4_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_exp["tN2O"] = world_sum_exp.ExportsFrom * n2o_factors["kg/tonne"][world_sum_exp.ghgCategory].values / 1000000
world_sum_imp["tCO2"] = world_sum_imp.ImportsFrom * co2_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tCH4"] = world_sum_imp.ImportsFrom * ch4_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp["tN2O"] = world_sum_imp.ImportsFrom * n2o_factors["kg/tonne"][world_sum_imp.ghgCategory].values / 1000000
world_sum_imp

c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\rlamboll\appdata\local\continuum\anaconda3\envs\silicone-5\lib\site-packages\ipykernel_launcher.py:4: Settin

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O
1,South Africa,"Coal; anthracite, whether or not pulverised, b...",3.542647e+08,3.542647e+08,0.0,3.681602e+07,9.622568e+00,Anthracite Coal,1.016112e+06,107.781320,1.562048e+01,Anthracite,9.298067e+05,94.588676,1.418830e+01
1,South Africa,"Coal; bituminous, whether or not pulverised, b...",1.273978e+09,1.273978e+09,0.0,1.470915e+09,8.661127e-01,Bituminous Coal,3.265062e+06,384.785744,5.617310e+01,Bitumen,4.132963e+06,512.139097,3.072835e+01
1,South Africa,"Coal; (other than anthracite and bituminous), ...",1.379110e+09,1.379110e+09,300727000.0,1.483721e+08,9.294940e+00,Sub-bituminous Coal,2.547882e+06,288.841048,4.256605e+01,Sub bituminous coal,2.504863e+06,260.651712,3.909776e+01
1,South Africa,"Briquettes, ovoids and similar solid fuels; ma...",4.443221e+07,4.443221e+07,0.0,5.693384e+07,7.804183e-01,Mixed Coal (Commercial Sector),9.874042e+04,11.509920,1.665265e+00,Other bituminous coal,1.084448e+05,11.463510,1.719527e+00
1,South Africa,"Lignite; whether or not agglomerated, excludin...",1.350000e+02,1.350000e+02,0.0,2.503600e+07,5.392235e-06,Lignite Coal,2.067010e-01,0.000023,3.422695e-06,Lignite,1.622565e-01,0.000016,2.409750e-06
1,South Africa,"Peat; (including peat litter), whether or not ...",5.674315e+07,5.674316e+07,15.0,5.718241e+07,9.923183e-01,Peat,5.598131e+04,16.012530,2.126664e+00,Peat,5.870420e+04,5.538132,7.753384e-01
1,South Africa,"Coke and semi-coke; of coal, lignite or peat, ...",8.016025e+08,8.016025e+08,68000.0,7.490820e+08,1.070113e+00,Coal Coke,2.490925e+06,241.228287,3.534480e+01,Coking coal,2.138451e+06,226.051900,3.390778e+01
1,South Africa,"Coal gas, water gas, producer gas and similar ...",1.533000e+03,1.533000e+03,0.0,0.000000e+00,1.533000e+23,nan,NaN,NaN,NaN,Gas coke,4.625674e+00,0.000216,4.323060e-06
1,South Africa,"Tar distilled from coal, from lignite, peat an...",1.217796e+06,1.217796e+06,0.0,1.324525e+06,9.194209e-01,nan,NaN,NaN,NaN,Coal tar,2.751732e+03,0.340983,5.114743e-02
1,South Africa,Oils and other products of the distillation of...,3.704628e+06,3.704632e+06,0.0,2.801721e+08,1.322269e-02,nan,NaN,NaN,NaN,Other petroleum products,1.091628e+04,1.489260,8.935563e-02


In [28]:
world_sum_exp.to_csv(join(output_folder, "detailed_exports.csv"))
report_exp = world_sum_exp.groupby(by="Country", axis=0).sum()
report_exp["GWP100 tCO2-equiv"] = report_exp.tCO2 + 28 * report_exp.tCH4 + 265 * report_exp.tN2O
report_exp.to_csv(join(output_folder, "total_exports_summary.csv"))
report_exp

,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio,tCO2_USA,tCH4_USA,tN2O_USA,tCO2,tCH4,tN2O,GWP100 tCO2-equiv
Country,,,,,,,,,,,,
Egypt,1.450403e+10,1.450403e+10,1.683730e+10,0.0,inf,8.393329e+05,87.196804,12.735905,4.365339e+07,5803.622692,334.000303,4.390440e+07
Nigeria,1.951365e+09,4.476203e+08,1.034462e+11,0.0,2.690085e+04,4.954951e+02,0.056172,0.008278,5.640320e+06,563.419953,18.922690,5.661110e+06
South Africa,8.736850e+10,8.195947e+10,6.931476e+10,1850404.0,2.491816e+01,2.083306e+08,24494.037676,3575.638653,2.794263e+08,34830.132034,2141.932480,2.809692e+08


In [29]:
world_sum_imp.to_csv(join(output_folder, "detailed_imports.csv"))
report_imp = world_sum_imp.groupby(by="Country", axis=0).sum()
report_imp["GWP100 tCO2-equiv"] = report_imp.tCO2 + 28 * report_imp.tCH4 + 265 * report_imp.tN2O
report_imp.to_csv(join(output_folder, "total_imports_summary.csv"))
report_imp

,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,tCO2_USA,tCH4_USA,tN2O_USA,tCO2,tCH4,tN2O,GWP100 tCO2-equiv
Country,,,,,,,,,,,,
Egypt,2.714328e+10,1.223704e+10,0.0,1.616417e+10,7.090300e+29,6.024155e+06,665.597377,97.203925,8.145651e+07,10168.196724,589.824581,8.189752e+07
Nigeria,8.635698e+09,6.252096e+07,0.0,1.250535e+10,1.087793e+02,7.297679e+03,0.801734,0.115653,2.634898e+07,3785.036589,227.118502,2.651515e+07
South Africa,3.585824e+10,3.585824e+10,301247673.0,9.259259e+09,1.533000e+23,9.474703e+06,1050.158874,153.496368,1.077972e+08,14174.033168,873.191599,1.084254e+08


In [30]:
report_net_exp = report_exp - report_imp
report_net_exp = report_net_exp[["GWP100 tCO2-equiv", "tCO2", "tCH4", "tN2O"]]
report_net_exp.to_csv(join(output_folder, "net_exports.csv"))
report_net_exp

,GWP100 tCO2-equiv,tCO2,tCH4,tN2O
Country,,,,
Egypt,-3.799312e+07,-3.780312e+07,-4364.574032,-255.824277
Nigeria,-2.085404e+07,-2.070866e+07,-3221.616636,-208.195812
South Africa,1.725437e+08,1.716291e+08,20656.098866,1268.740881


# Sanity check against Nigeria's oil exports

In [31]:
# Net exports were 1979 barrels per day: https://www.ceicdata.com/en/indicator/nigeria/crude-oil-exports
ReportedNigerianOilExport = 140*1979*365
# And net imports: https://www.eia.gov/dnav/pet/pet_move_neti_dc_NUS-NNI_mbblpd_a.htm
ReportedNigerianNetOilImport = 140*177000*365

In [32]:
NigerianOilExp = world_sum_exp[(world_sum_exp.Country=="Nigeria") & (world_sum_exp.ghgCategory.isin(
    ["Crude oil", "Motor gasoline", "Other petroleum products"]
))]
NigerianOilExp

,Country,Commodity,ExportsFrom,World,ImportsTo,ReExportsFrom,ExpFromImpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O
1375,Nigeria,Oils and other products of the distillation of...,170352700.0,170352700.0,6.333000e+03,0.0,26899.210485,nan,NaN,NaN,NaN,Other petroleum products,501971.486982,68.481785,4.108907
1424,Nigeria,Petroleum oils and oils obtained from bitumino...,0.0,NaN,8.879210e+10,0.0,0.000000,nan,NaN,NaN,NaN,Crude oil,0.000000,0.000000,0.000000
1478,Nigeria,Petroleum oils and oils from bituminous minera...,276999420.0,276999420.0,1.386693e+09,0.0,0.199755,nan,NaN,NaN,NaN,Motor gasoline,850385.449406,122.710743,7.362645


In [33]:
sum(NigerianOilExp.ExportsFrom) / ReportedNigerianOilExport

4.423670853155787

In [34]:
NigerianOilImp = world_sum_imp[(world_sum_imp.Country=="Nigeria") & (world_sum_imp.ghgCategory.isin(
    ["Crude oil", "Motor gasoline", "Other petroleum products"]
))]
NigerianOilImp

,Country,Commodity,ImportsFrom,World,ReImportsFrom,ExportsTo,ImpFromExpToRatio,emissionsCommodity,tCO2_USA,tCH4_USA,tN2O_USA,ghgCategory,tCO2,tCH4,tN2O
1,Nigeria,Oils and other products of the distillation of...,3.818150e+06,3818151.0,0.0,2.812205e+06,1.357707,nan,NaN,NaN,NaN,Other petroleum products,1.125079e+04,1.534896,0.092094
1,Nigeria,Petroleum oils and oils obtained from bitumino...,0.000000e+00,0.0,0.0,4.012510e+05,0.000000,nan,NaN,NaN,NaN,Crude oil,0.000000e+00,0.000000,0.000000
1,Nigeria,Petroleum oils and oils from bituminous minera...,8.448020e+09,0.0,0.0,1.207995e+10,0.699342,nan,NaN,NaN,NaN,Motor gasoline,2.593534e+07,3742.472755,224.548365


In [35]:
(sum(NigerianOilImp.ImportsFrom) - sum(NigerianOilExp.ExportsFrom)) / ReportedNigerianNetOilImport

0.8849918509182173

The sanity check indicates that the difference of the imports and exports is more robust than either individually, possibly due to transport of oil through Nigeria